In [1]:
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
from typing import Any
from google.oauth2 import service_account
import logging


In [2]:
credentials = service_account.Credentials.from_service_account_file("connection-123-892e002c2def.json")

In [6]:

destination_table = "btc_rsi"

logger = logging.getLogger(__name__)

def calculate_rsi(credentials) -> pd.DataFrame:
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)

    query = """
    SELECT
    timestamp,
    price
    FROM connection-123.signals.bitcoin_price
    WHERE DATE(timestamp) < DATE(CURRENT_TIMESTAMP())

    ORDER BY 1
    """
    query_job = client.query(query)
    results = query_job.result()
    df = results.to_dataframe()

    bytes_processed = query_job.total_bytes_processed
    logger.info(f"Query processed {bytes_processed:,} bytes ({bytes_processed / 1024 / 1024:.2f} MB)")

    # Calculate price changes
    df['price_change'] = df['price'].diff()

    # Separate gains and losses
    df['gain'] = df['price_change'].where(df['price_change'] > 0, 0)
    df['loss'] = (-df['price_change']).where(df['price_change'] < 0, 0)

    # Calculate 14-period average gains and losses using exponential moving average
    df['avg_gain'] = df['gain'].ewm(span=14, adjust=False).mean()
    df['avg_loss'] = df['loss'].ewm(span=14, adjust=False).mean()

    # Calculate RSI
    df['rs'] = df['avg_gain'] / df['avg_loss']
    df['rsi_14'] = 100 - (100 / (1 + df['rs']))

    # Return timestamp and RSI
    rsi_result = df[['timestamp', 'rsi_14']].copy()

    return rsi_result


In [7]:
data = calculate_rsi(credentials)

In [8]:
print(data)

      timestamp     rsi_14
0    2024-08-11        NaN
1    2024-08-12   0.000000
2    2024-08-13  23.212239
3    2024-08-14  52.421453
4    2024-08-15  31.708875
..          ...        ...
361  2025-08-07  46.127705
362  2025-08-08  59.736348
363  2025-08-09  54.676633
364  2025-08-10  53.474564
365  2025-08-10  64.522740

[366 rows x 2 columns]
